In [43]:
### Import das bibliotecas necessarias
import pandas as pd
#import pyodbc
from sqlalchemy import create_engine

In [44]:
# Conexão com o banco de dados
try:
    server_name = 'DESKTOP-IBC02BG'
    database_name = 'dbProjetoImpacta'
    conn_str = f'mssql+pyodbc://{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server'
    engine = create_engine(conn_str)
    print('Conectado com sucesso!')
    
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Conectado com sucesso!


In [45]:
# API banco central cotação dolar
path = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='01-01-2020'&@dataFinalCotacao='12-31-2025'&$top=9000&$format=text/csv&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao"
# Leitura dos dados e criação do dataframe
tabela = pd.read_csv(path)
# Renomeando as colunas do dataframe
tabela = tabela.rename(columns={'cotacaoCompra': 'Cotacao de Compra','cotacaoVenda':'Cotacao de Venda','dataHoraCotacao':'Data e Hora da Cotacao'})
# Separando a tabela data e hora em duas tabelas
tabela[['Data', 'Hora']] = tabela['Data e Hora da Cotacao'].str.split(' ', 1, expand=True)
# Apagando a coluna com data e hora juntos
tabela.drop(columns=['Data e Hora da Cotacao'], inplace=True)
tabela.drop(columns=['Hora'], inplace=True)

In [46]:
# Converter os tipos e organizar os dados das colunas 

tabela['Cotacao de Compra'] = tabela['Cotacao de Compra'].astype(str).str.replace(',', '.').astype(float)
tabela['Cotacao de Venda'] = tabela['Cotacao de Venda'].astype(str).str.replace(',', '.').astype(float)
tabela['Data'] = pd.to_datetime(tabela['Data'], format='%Y-%m-%d')
tabela['Data'] = tabela['Data'].dt.strftime('%Y-%m-%d')

tabela['Data'] = pd.to_datetime(tabela['Data'])




display(tabela)

,Cotacao de Compra,Cotacao de Venda,Data
0,4.0207,4.0213,2020-01-02
1,4.0516,4.0522,2020-01-03
2,4.0548,4.0554,2020-01-06
3,4.0835,4.0841,2020-01-07
4,4.0666,4.0672,2020-01-08
...,...,...,...
821,5.0828,5.0834,2023-04-10
822,5.0122,5.0128,2023-04-11
823,4.9490,4.9496,2023-04-12
824,4.9091,4.9097,2023-04-13


In [47]:
print(tabela.dtypes)

Cotacao de Compra           float64
Cotacao de Venda            float64
Data                 datetime64[ns]
dtype: object


In [48]:
try:
    # Salvando o DataFrame no banco de dados MySQL
    tabela.to_sql('cotacao', con=engine, if_exists='replace', index=False)
    print('Dados gravados no banco de dados...')


except Exception as e:
    print(f"Ocorreu um erro: {e}")

Dados gravados no banco de dados...


In [49]:
# Executa a consulta SQL para contar as linhas da tabela
result = engine.execute('SELECT COUNT(*) FROM cotacao')
count = result.fetchone()[0]

print(f'O número de linhas na tabela é: {count}')

O número de linhas na tabela é: 826


In [50]:
# Fechando a conexão com o banco de dados
engine.dispose()